### 참고자료
https://seongjaemoon.github.io/python/2018/05/16/python-course6.html

#### 크롬 웹드라이버
https://sites.google.com/a/chromium.org/chromedriver/downloads


In [90]:
import time

# 크롬 브라우저 조작을 위한 드라이버
from selenium import webdriver


# 페이지 스크롤링을 위한 모듈
from selenium.webdriver.common.keys import Keys

# 다운로드 받은 드라이버 주소
DRIVER_DIR = 'driver2/chromedriver'

In [91]:
# 드라이버를 이용해 크롬 브라우저를 실행시켜 조작하기 위해 설정 
driver = webdriver.Chrome(DRIVER_DIR)

# 암묵적으로 웹 자원을 (최대) 3초 기다리기
driver.implicitly_wait(3) 

### Kohler

In [93]:
# 목표로 하는 url의 내용을 가져온다. 
url = "https://www.us.kohler.com/us/browse/kitchen-kitchen-sinks/_/N-25b4"
driver.get(url)

In [94]:
# 끝까지 스크롤다운
# https://www.edureka.co/community/4578/possible-scroll-webpage-selenium-webdriver-programmed-python
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#### 멀티 클래스 element 가져오기
https://stackoverflow.com/questions/37771604/selenium-compound-class-names-not-permitted

In [98]:
# 상품 이름
item_list = driver.find_elements_by_css_selector('.product-panel__summary.product-panel__summary-new')


item = []

for i in item_list:
    item.append(i.text)
    
len(item)

267

In [99]:
# 가격

price_list = driver.find_elements_by_css_selector('.product-panel__price.product-panel__price-new.light-gray--sku--price.promo-price-discountprice-nk')


price = []

for p in price_list:
    price.append(p.text)
    
len(price)

267

In [100]:
# 모델명
sku_list = driver.find_elements_by_css_selector('.product-panel__sku.light-gray--sku--price.promo-price-sku-nk')


sku = []

for s in sku_list:
    sku.append(s.text)
    
len(sku)

267

In [147]:
#https://cmdlinetips.com/2018/01/how-to-create-pandas-dataframe-from-multiple-lists/
import pandas as pd

data_tuples = list(zip(item, sku, price))
data_tuples
df = pd.DataFrame(data_tuples, columns=['item', 'sku', 'price'])

df.head()


item  \
0           Ballad 31-1/2" x 17-7/8" x 9" under-mount single-bowl kitchen sink   
1  Kennon 33" x 22" x 9-5/8" Neoroc top-/under-mount double-equal kitchen s...   
2          Ballad 31-1/2" x 17-3/4" x 9" under-mount double-equal kitchen sink   
3                                              Farmstead utility rack with cup   
4                Riverby 25" x 22" x 5-7/8" top-mount single-bowl kitchen sink   

            sku    price  
0  K-RH20061-NA  $399.00  
1  K-8185-1-CM1  $435.00  
2  K-RH20062-NA  $399.00  
3    K-21109-ST   $90.00  
4    K-5479-4-0  $599.00

In [148]:
pd.set_option('max_colwidth', 90)
df.sort_values('price').head()

item  \
186  Undertone 31-1/2" x 18" x 9-3/4" under-mount double-equal bowl kitchen s...   
62   Strive 35" x 18-5/16" x 9-5/16" under-mount extra-large single bowl kitc...   
19   Prolific 29" x 17-3/4" x 10-15/16" under-mount single-bowl kitchen sink ...   
63   Strive 32" x 18-5/16" x 9-5/16" Smart Divide under-mount large/medium do...   
66   Strive 35-1/2" x 20-1/4" x 9-5/16" under-mount extra-large/medium double...   

            sku      price  
186   K-3351-NA  $1,006.95  
62    K-5283-NA  $1,009.00  
19   K-23651-NA  $1,029.00  
63    K-5284-NA  $1,029.00  
66    K-5282-NA  $1,030.00

# Preprocessing

In [103]:
df.head()

item  \
0           Ballad 31-1/2" x 17-7/8" x 9" under-mount single-bowl kitchen sink   
1  Kennon 33" x 22" x 9-5/8" Neoroc top-/under-mount double-equal kitchen s...   
2          Ballad 31-1/2" x 17-3/4" x 9" under-mount double-equal kitchen sink   
3                                              Farmstead utility rack with cup   
4                Riverby 25" x 22" x 5-7/8" top-mount single-bowl kitchen sink   

            sku    price  
0  K-RH20061-NA  $399.00  
1  K-8185-1-CM1  $435.00  
2  K-RH20062-NA  $399.00  
3    K-21109-ST   $90.00  
4    K-5479-4-0  $599.00

In [149]:
# 가격 전처리
df['price'] = df['price'].str.replace('$','')
df['price'] = df['price'].str.replace(',','')

print(pd.isna(df['price']).sum())

0


In [150]:
df['price'] = pd.to_numeric(df['price'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 3 columns):
item     267 non-null object
sku      267 non-null object
price    267 non-null float64
dtypes: float64(1), object(2)
memory usage: 6.3+ KB


In [151]:
def find_sink_type(x):
    if 'kitch' in x:
        return 'kitchen sink'
    elif 'bar sink' in x:
        return 'bar sink'
    elif 'utility sink' in x:
        return 'utility sink'
    elif 'cutting board' in x:
        return 'cutting board'
    elif 'leg' in x:
        return 'leg'
    elif 'rack' in x:
        return 'rack'
    else:
        return 'kitchen sink'
    
    
    
df['sink_type'] = df['item'].apply(find_sink_type)  

pd.unique(df['sink_type'])

array(['kitchen sink', 'rack', 'cutting board', 'bar sink', 'leg',
       'utility sink'], dtype=object)

In [155]:
# 싱크만 필터링하기
df.loc[df['sink_type'] == 'kitchen sink']

df = df.loc[(df['sink_type']=='kitchen sink') | (df['sink_type']=='bar sink')]

df['sink_type'].value_counts()

kitchen sink    226
bar sink         20
Name: sink_type, dtype: int64

In [156]:
# 브랜드
df['brand'] = df['item'].str.split(' ').str[0]
df.head()

item  \
0           Ballad 31-1/2" x 17-7/8" x 9" under-mount single-bowl kitchen sink   
1  Kennon 33" x 22" x 9-5/8" Neoroc top-/under-mount double-equal kitchen s...   
2          Ballad 31-1/2" x 17-3/4" x 9" under-mount double-equal kitchen sink   
4                Riverby 25" x 22" x 5-7/8" top-mount single-bowl kitchen sink   
5  Farmstead 45" x 25" x 9" top-mount/wall-mount kitchen sink with three fa...   

              sku   price     sink_type      brand size_width  
0    K-RH20061-NA   399.0  kitchen sink     Ballad    31-1/2"  
1    K-8185-1-CM1   435.0  kitchen sink     Kennon        33"  
2    K-RH20062-NA   399.0  kitchen sink     Ballad    31-1/2"  
4      K-5479-4-0   599.0  kitchen sink    Riverby        25"  
5  K-21103-3HP5-0  3150.0  kitchen sink  Farmstead        45"

In [171]:
# 사이즈
df['size_width'] = df['item'].str.split("\"").str[0]
df['size_width'] = df['size_width'].str.split(' ').str[1]
df['size_width'].value_counts().sort_values(ascending=False)

33               94
25               18
Self-Trimming    14
31-1/2           10
15                9
32                8
45                7
35-3/4            6
Preserve          4
31                4
27                4
Chef              3
29-3/4            3
33-1/2            3
35-1/2            3
Degree            3
23                3
24                3
36                2
24-1/4            2
43                2
31-5/8            2
29                2
20                2
15-1/4            2
18                2
15-1/2            2
15-3/4            2
Lace              1
12-1/4            1
17                1
16-1/4            1
19-5/8            1
31-1/4            1
Fino              1
18-1/2            1
27-1/2            1
24-1/2            1
29-1/2            1
30-3/4            1
29-11/16          1
41-5/8            1
34                1
Dory              1
44                1
20-7/8            1
22                1
25-1/4            1
18-3/4            1
30                1


0         Ballad 31-1/2
1             Kennon 33
2         Ballad 31-1/2
4            Riverby 25
5          Farmstead 45
7         Ballad 31-1/2
8          Cairn 24-1/2
9           Prolific 44
10             Vault 24
11         Farmstead 45
12             Verse 33
13             Verse 33
14         Cairn 27-1/2
15            Kennon 33
17         Farmstead 45
18             Lyric 15
19          Prolific 29
20         Farmstead 45
21             Verse 33
23          Prolific 23
24         Cairn 15-1/2
25            Kennon 33
28        Iron/Tones 33
30           Riverby 33
31         Farmstead 45
32         Farmstead 45
33           Riverby 25
35             Verse 33
36             Verse 33
37             Verse 33
             ...       
235    Undertone 15-1/2
236    Undertone 10-3/4
237            Vault 33
238            Vault 33
239            Vault 33
240            Vault 25
241            Vault 25
242            Vault 25
243         Staccato 25
244         Staccato 25
245          Toc

# bs로 해보기
https://devyurim.github.io/python/crawler/2018/08/13/crawler-4.html

In [ ]:
html = driver.page_source
type(html)

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

#print(soup.text)
print(soup.prettify())


<!DOCTYPE html>
<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Prep sink - All Questions - Home Design Discussions
  </title>
  <meta content="View discussions on home design related topics such as installation costs, popular appliance brands, DIY repairs and All Questions - prep sink" name="description"/>
  <meta charset="utf-8"/>
  <meta content="width=1024" name="viewport"/>
  <meta content="origin-when-cross-origin" name="referrer"/>
  <link href="//as.hzcdn.com/" rel="dns-prefetch"/>
  <link href="//st.hzcdn.com/simgs/" rel="dns-prefetch"/>
  <link href="//st.hzcdn.com/fimgs/" rel="dns-prefetch"/>
  <link crossorigin="anonymous" href="//as.hzcdn.com/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//st.hzcdn.com/simgs/" rel="preconnect"/>
  <link crossorigin="anonymous" href="//st.hzcdn.com/fimgs/" rel="preconnect"/>
  <link class="css-httpssthzcdncomres3961cssstylefhouzzbootstrapuihouzzThemehouzzresponsive1425px" href="https://st.hzcdn.com/re

In [10]:
#titles = soup.select('hz-question-card__title')


#for title in titles:
#    print(title.text)

In [17]:
# 글 제목
titles = driver.find_elements_by_class_name('hz-question-card__title')

tmp = []

for title in titles:
    tmp.append(title.text)
    
tmp

['Prep sink or no prep sink',
 'Prep sink or no prep sink in island?',
 'Prep sink location or no prep sink',
 'Prep sink or no prep sink?',
 'Prep sink or no prep sink',
 'Please help with prep sink cabinet? How many inches should I have from edge of prep sink to counter edge?',
 'Size of prep sink and prep area',
 'Regular or prep faucet for prep sink',
 'Sinks! Drain location, Julien vs CreateGoodSinks, & prep sink question',
 'Prep Sink, Cleanup Sink, Workstation Sink & garbage disposal question?',
 'Kitchen layout - single sink vs. main sink + prep sink',
 'Faucet sizes: Sink vs prep sink.... ?',
 'Primary sink (not prep sink) in Island - yay or nay?',
 'is it ok to have prep sink different from main sink?',
 'Need prep sink to go with shaw 30" fireclay farmhouse sink',
 'Kitchen sink and prep sink',
 'Do I need a sink rack in a cast iron prep sink?',
 'Sink and prep sink in same island',
 'Prep sink to coordinate with shaw sink',
 'Matching of kitchen sink & prep sink',
 'Would y

In [19]:
# 글 제목
date_list = driver.find_elements_by_class_name('hz-question-card__publish-date-label')

date = []

for d in date_list:
    date.append(d.text)
    
date

['January 10, 2019',
 'December 17, 2018',
 'December 4, 2017',
 'January 13, 2017',
 'August 19, 2015',
 'December 30, 2012',
 'March 10, 2015',
 'April 26, 2013',
 'February 15, 2018',
 'June 4, 2018',
 'January 18, 2013',
 'August 11, 2017',
 'August 26, 2016',
 'February 4, 2016',
 'September 16, 2015',
 'December 16, 2015',
 'October 28, 2015',
 'August 11, 2015',
 'April 29, 2015',
 'January 16, 2014',
 'February 3, 2013',
 'February 9, 2013',
 'January 12, 2013',
 'January 22, 2013',
 'April 30, 2013',
 'April 19, 2012',
 'October 1, 2012',
 'May 11, 2011',
 'July 1, 2011',
 'May 12, 2011']

In [ ]:
driver.close()